# Measuring the impact of student research labs at the University of Texas 

In [5]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
import janitor 
import statsmodels.formula.api as smf
import censusdata 
%matplotlib inline
pd.set_option('display.max_colwidth', 100)

In [3]:
df = pd.read_excel("IDA Student Info Fall 2018 - Test.xlsx", sheet_name= "Sheet1").clean_names()

pd.set_option("display.max_columns", 101)
pd.set_option("display.max_rows", 10)
pd.set_option('display.min_rows', 10)
pd.set_option('display.max_colwidth', 25)


# Codebook 
### \*Currently missing columns with individual identifiers\*
| **Variable Name** | **Plain English**|
| --- | --- |
|sch_maj1| School Major |
|ut_abbr1| UT abbreviation |
|cola_abbr1| COLA abbreviation|
|profiles| There's a lot of information in this column that I don't think is important|
|maj1_name| Name of major |
|sch_maj2| Second major|
|ut_abbr2| UT Abbreviation of second major |
|cola_abbr2| COLA abbreviation of second major|
|maj2_name|Second Major Name|
|cumulative_gpa |Cumulative GPA|
|gpa_hrs | GPA hours completed at UT (includes currently taking)|
|credit_hrs_excludes_current_ |Excludes currently taking, but it includes transfer credits|
|current_hrs|Current hours taking this semester|
|perm_street_2|Second Address|
|perm_street_3|Third Address|
|perm_city| Permanent City|
|perm_state| Permanent State|
|perm_postal_code| Postal Code|
|citizenship| Citizenship |
|gender|gender|
|ethnicities| Ethnicities|
|classification| Freshman/Sophmore etc.|
|ut_degrees| UT degrees if they've already been awarded one. (I think)|
|current_ageas_of_today| Current Age as of last draw of the data|
|cumulative_gpa_in_residenceunavailable_if_current_ccyys| Cumulative GPA in residence (unavailable if taking classes this semester)|
|cumulative_hrs_incomplete|Cumulative hours completed|
|cumulative_hrs_transferred|Cumulative hours transfered|
|sem_gpa_fall_2018unavailable_if_current_ccyys|Semester GPA Fall 2018 (unavailable if current semester)|
|sem_gpa_hrs_fall_2018unavailable_if_current_ccyys|Semester GPA and Hours Fall 2018 (unavailable if current semester)|
|overall_act_score| ACT Score|
|sat_verbal_score| SAT Verbal Score |
|sat_quant_score| SAT Quant Score |
|semester_hrs_incomplete_fall_2018_unavailable_if_current_ccyys| Semester hours incomplete (unavailable if current semester)|
|fatherseducation|Father's education|
|motherseducation|Mother's education|
|familyincome|Family income category|
|high_school|High School|
|hs_rank | High School Rank|
|unofficial_hnrs_pgm_s_ |Unofficial Honors Program|

# Data Cleaning 

#### UT data

In [4]:
df = df.drop(['sch_maj2', 'ut_abbr2','cola_abbr2','maj2_name','cumulative_gpa_in_residenceunavailable_if_current_ccyys',
             'sem_gpa_fall_2018unavailable_if_current_ccyys','sem_gpa_hrs_fall_2018unavailable_if_current_ccyys',
             'unofficial_hnrs_pgm_s_','semester_hrs_incomplete_fall_2018_unavailable_if_current_ccyys','cumulative_hrs_incomplete', 'ut_degrees', 'cola_abbr1',
              'perm_street_3','perm_street_2','profiles'], axis=1)

df = df.rename(columns={"credit_hrs_excludes_current_": "credit_hrs_ex_cur", "current_ageas_of_today": "age", "cumulative_hrs_transferred" : "hrs_transferred"})



df['hs_rank'] = df['hs_rank'].astype(str)
df['overall_act_score'] = df['overall_act_score'].astype(str)
df['age'] = df['age'].astype(str)
df['sat_verbal_score'] = df['sat_verbal_score'].astype(str)
df['sat_quant_score'] = df['sat_quant_score'].astype(str)

df['age'] = df['age'].str.replace("\(R\)", "")
df['age'] = df['age'].str.replace("\*Minor", "")

df['classification'] = df['classification'].replace("\(R\)", "")
df = df.replace('unknown', "0/0")
df = df.replace('None', "0")

df['sat_verbal_score'] = df['sat_verbal_score'].astype(int)
df['sat_quant_score'] = df['sat_quant_score'].astype(int)
df['overall_act_score'] = df['overall_act_score'].astype(int)
df['age'] = df['age'].astype(int)

df[['numerator','denominator']] = df.hs_rank.str.split("/",expand=True,)
df['numerator'] = df['numerator'].astype(int)
df['denominator'] = df['denominator'].astype(int)
df['hs_rank'] = df['numerator']/df['denominator']
df['hs_rank'] = df['hs_rank']*100
df = df.drop(['numerator','denominator'], axis = 1)

df

,sch_maj1,ut_abbr1,maj1_name,cumulative_gpa,gpa_hrs,credit_hrs_ex_cur,current_hrs,perm_city,perm_state,perm_postal_code,citizenship,gender,ethnicities,classification,age,hrs_transferred,overall_act_score,sat_verbal_score,sat_quant_score,fatherseducation,motherseducation,familyincome,high_school,hs_rank
0,L 23400,C C,Classical Studies,3.1717,68,77,18,Lampasas,TX,76550-3612,U. S. Citizen,Male,Black or African Amer...,Junior,20,0,24,550,600,High School Diploma O...,Some College,0/0,Lampasas High School ...,6.048387
1,L 50600 (R),HIS,History,2.0767,43,76,17 (R),Austin,TX,78728-4418,U. S. Citizen,Female,Asian,Junior (R),21,35,23,560,590,Graduate Or Professio...,Graduate Or Professio...,H,Westwood High School ...,69.798658
2,L 00300,NaN,College of Liberal Ar...,4.0000,3,3,3,Austin,TX,78704-7957,U. S. Citizen,Female,Hispanic,Senior,32,0,0,0,0,Bachelor's Or Four-ye...,Bachelor's Or Four-ye...,C,Homer Hanna High Scho...,NaN
3,L 55000,IRG,International Relatio...,3.5208,34,79,15,Spring,TX,77388-5765,U. S. Citizen,Female,"Hispanic, Asian",Junior,20,42,22,660,540,Graduate Or Professio...,Associate's Degree,G,Klein Collins High Sc...,45.080946
4,L 47500,GOV,Government,0.0000,0,0,13,Lewisville,TX,75057-5123,U. S. Citizen,Female,Black or African Amer...,Freshman,18,0,0,700,600,Bachelor's Or Four-ye...,Educational Level Unk...,A,Lewisville High Schoo...,7.200720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10594,L 50600,HIS,History,2.4015,20,47,13,Edinburg,TX,78541-2287,U. S. Citizen,Female,"Hispanic, White",Sophomore,19,18,0,630,650,No High School,Associate's Degree,B,Edinburg North High S...,1.423488
10595,L 50600,HIS,History,4.0000,12,77,12,Austin,TX,78746-7806,U. S. Citizen,Female,White,Junior,19,45,31,0,0,Graduate Or Professio...,Bachelor's Or Four-ye...,I,Westlake High School ...,NaN
10596,L 55000,IRG,International Relatio...,2.8000,15,43,12,Austin,TX,78744-3644,U. S. Citizen,Female,Hispanic,Sophomore,19,0,24,0,0,Some High School,No High School,A,Kipp Austin Collegiat...,20.289855
10597,5 70700,ARH,Art History (Bachelor...,3.0738,81,112,18,Short Hills,NJ,070781653,U. S. Citizen,Female,White,Senior,21,22,0,600,570,Bachelor's Or Four-ye...,Bachelor's Or Four-ye...,G,Millburn High School ...,NaN


### The Ethnicities column provides an exhaustive list of an individual's ethnicities. Below I changed the values of individuals with multiple ethnicities to "mixed race". 

In [5]:
df.loc[df['ethnicities'] == 'Hispanic, Asian', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Asian, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, Black or African American', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Black or African American, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Asian, Black or African American', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, American Indian or Alaska Native', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Asian, Native Hawaiian or Other Pacific Islander', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, Black or African American, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Native Hawaiian or Other Pacific Islander, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'American Indian or Alaska Native, Black or African American, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'American Indian or Alaska Native, Black or African American','ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Asian, Black or African American, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Asian, Native Hawaiian or Other Pacific Islander, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, Asian, Black or African American', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, American Indian or Alaska Native, Black or African American', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, American Indian or Alaska Native, Asian, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, American Indian or Alaska Native, Black or African American, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, American Indian or Alaska Native, Asian, Black or African American, Native Hawaiian or Other Pacific Islander, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'American Indian or Alaska Native, Asian, Black or African American', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, Asian, Native Hawaiian or Other Pacific Islander', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, American Indian or Alaska Native, Asian', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, Asian, Native Hawaiian or Other Pacific Islander, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'American Indian or Alaska Native, Asian, Native Hawaiian or Other Pacific Islander', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Black or African American, Native Hawaiian or Other Pacific Islander', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, Native Hawaiian or Other Pacific Islander, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'American Indian or Alaska Native, Asian, Black or African American, Native Hawaiian or Other Pacific Islander, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'American Indian or Alaska Native, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, American Indian or Alaska Native, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'American Indian or Alaska Native, Asian, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, Asian, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, Native Hawaiian or Other Pacific Islander', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, Asian', 'ethnicities'] = "Mixed Race"

### Census Data on city/town population 

In [15]:
pop = pd.read_csv('sub-est2018_all.csv', encoding='latin-1').clean_names()
pop.head()

texas = pop[pop['stname'] == 'Texas']
texas

,name,stname,popestimate2018
71188,Texas,Texas,28701845
71189,Abbott city,Texas,367
71190,Abernathy city,Texas,2724
71191,Abilene city,Texas,122999
71192,Ackerly city,Texas,227
...,...,...,...
74301,Balance of Young County,Texas,5671
74302,Zapata County,Texas,14190
74303,Zavala County,Texas,11983
74304,Crystal City city,Texas,7269


### Downloading Census Data 

In [17]:
#urban_areas = censusdata.download('acs5', 2015, censusdata.censusgeo([('urban area', '*')]),
                             #      ['B01001_001E'])

#urban_areas 

# Statistical Analysis 

In [12]:
model = smf.ols("cumulative_gpa ~ citizenship + age + sat_verbal_score + sat_quant_score + hs_rank + motherseducation + fatherseducation + ethnicities", df)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         cumulative_gpa   R-squared:                       0.120
Model:                            OLS   Adj. R-squared:                  0.117
Method:                 Least Squares   F-statistic:                     37.99
Date:                Tue, 24 Mar 2020   Prob (F-statistic):          1.69e-185
Time:                        11:27:06   Log-Likelihood:                -12792.
No. Observations:                7522   AIC:                         2.564e+04
Df Residuals:                    7494   BIC:                         2.583e+04
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
============================================================================================================================
                                                               coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------------
Intercept                                                   -0.4088      0.359     -1.138      0.255      -1.113       0.296
citizenship[T.Permanent Resident Alien]                     -0.2062      0.187     -1.103      0.270      -0.573       0.160
citizenship[T.U. S. Citizen]                                -0.2026      0.182     -1.114      0.265      -0.559       0.154
motherseducation[T.Associate's Degree]                      -0.5972      0.782     -0.763      0.445      -2.131       0.936
motherseducation[T.Bachelor's Or Four-year Degree]          -0.5704      0.781     -0.731      0.465      -2.101       0.960
motherseducation[T.Educational Level Unknown]               -1.3347      0.788     -1.694      0.090      -2.879       0.210
motherseducation[T.Graduate Or Professional Degree]         -0.5525      0.781     -0.708      0.479      -2.083       0.978
motherseducation[T.High School Diploma Or Equivalent]       -0.7448      0.782     -0.953      0.341      -2.277       0.787
motherseducation[T.No High School]                          -0.9487      0.785     -1.208      0.227      -2.488       0.590
motherseducation[T.Some College]                            -0.6979      0.781     -0.894      0.371      -2.228       0.832
motherseducation[T.Some High School]                        -1.0395      0.782     -1.330      0.184      -2.572       0.493
fatherseducation[T.Associate's Degree]                       1.4660      0.777      1.886      0.059      -0.057       2.989
fatherseducation[T.Bachelor's Or Four-year Degree]           1.5201      0.775      1.962      0.050       0.001       3.039
fatherseducation[T.Educational Level Unknown]                2.0836      0.783      2.661      0.008       0.549       3.618
fatherseducation[T.Graduate Or Professional Degree]          1.6324      0.774      2.108      0.035       0.114       3.151
fatherseducation[T.High School Diploma Or Equivalent]        1.4942      0.775      1.927      0.054      -0.025       3.014
fatherseducation[T.No High School]                           1.6313      0.779      2.095      0.036       0.105       3.158
fatherseducation[T.Some College]                             1.6239      0.775      2.095      0.036       0.104       3.143
fatherseducation[T.Some High School]                         1.6814      0.777      2.164      0.030       0.158       3.205
ethnicities[T.Asian]                                        -0.7943      0.471     -1.685      0.092      -1.718       0.130
ethnicities[T.Black or African American]                    -0.9174      0.475     -1.933      0.053      -1.848       0.013
ethnicities[T.Hispanic]                                     -0.9222      0.474     

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10599 entries, 0 to 10598
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sch_maj1           10599 non-null  object 
 1   ut_abbr1           10088 non-null  object 
 2   maj1_name          10599 non-null  object 
 3   cumulative_gpa     10599 non-null  float64
 4   gpa_hrs            10599 non-null  int64  
 5   credit_hrs_ex_cur  10599 non-null  int64  
 6   current_hrs        10599 non-null  object 
 7   perm_city          10584 non-null  object 
 8   perm_state         10192 non-null  object 
 9   perm_postal_code   10529 non-null  object 
 10  citizenship        10599 non-null  object 
 11  gender             10599 non-null  object 
 12  ethnicities        9834 non-null   object 
 13  classification     10599 non-null  object 
 14  age                10599 non-null  int32  
 15  hrs_transferred    10599 non-null  int64  
 16  overall_act_score  105